Some words

In [1]:
import os

import numpy as np

from tti_explorer import case, config, contacts

In [2]:
def print_doc(func):
    print(func.__doc__)

Before we do anything, let's make a random state

In [3]:
rng = np.random.RandomState(0)

We will first do a short tour of the functionality, then show how this is put together to generate simulation results.

## Generate a case

The function we use for this is `simulate_case` in `case.py`

In [4]:
print_doc(case.simulate_case)

simulate_case

    Args:
        rng (np.random.RandomState): random number generator.
        p_under18 (float): Probability of case being under 18
        infection_proportions (list[float]): Probs of being symp covid neg, symp covid pos, asymp covid pos
        p_day_noticed_symptoms (np.array[float]): Distribution of day on which case notices
            their symptoms. (In our model this is same as reporting symptoms.)
            Conditional on being symptomatic.
        inf_profile (list[float]): Distribution of initial exposure of positive secondary cases
            relative to start of primary case's infectious period.

    Returns (Case): case with attributes populated.
    


We store our config values in `config.py`. You can retrieve them as follows

In [5]:
case_config = config.get_case_config("oxteam")
case_config.keys()

dict_keys(['p_under18', 'infection_proportions', 'p_day_noticed_symptoms', 'inf_profile'])

We use these parameters to simulate a case

In [6]:
primary_case = case.simulate_case(rng, **case_config)
print_doc(primary_case)

Case(under18, covid, symptomatic, day_noticed_symptoms, inf_profile)


Returned is a `namedtuple`, `case.Case`, with stochastically generated attributes.

In [7]:
rng = np.random.RandomState(0)

### Deeper: Case attributes

_todo BE: explain case attributes_

## Generate contacts
Social contacts are also represented as a `namedtuple`, called `Contacts` and defined in `contacts.py`.

To simulate social contacts, we use the BBC Pandemic Dataset. This is stratified as over/under 18 to give different patterns of social contact depending on the age of the case. 

In [8]:
def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [9]:
path_to_bbc_data = os.path.join(os.pardir, "data", "bbc-pandemic")

over18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_o18.csv"))
under18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_u18.csv"))

Now that we have the data loaded, we use `EmpiricalContactsSimulator` to sample these tables for contacts of the primary case, then simulate their infection under a no measures scenario (i.e. no government intervention)

In [10]:
print_doc(contacts.EmpiricalContactsSimulator.__init__)

Simulate social contact using the BBC Pandemic dataset

        Args:
            over18 (np.array[int]): Contact data for over 18s.
            under18 (np.array[int]): Contact data for under 18s.
            rng (np.random.RandomState): Random state.

        


In [11]:
simulate_contacts = contacts.EmpiricalContactsSimulator(over18, under18, rng)

We can now use the callable `simulate_contacts` to simulate social contacts of the primary case

In [12]:
print_doc(simulate_contacts.__call__)

Generate a social contact for the given case.

        A row from the table corresponding to the age of the `case` is sampled
        uniformly at random. A contact is generated with daily contacts as
        given by that row. These contacts are infected at random with attack rates
        given by the SARs and whether or not the `case` is symptomatic. If the
        `case` is COVID negative, then no contacts are infected.

        Args:
            case (Case): Primary case.
            home_sar (float): Secondary attack rate for household contacts. (Marginal
            probability of infection over the whole simulation).
            work_sar (float): Secondary attack rate for contacts in the work category.
            other_sar (float): Secondary attack rate for contacts in the other category.
            asymp_factor (float): Factor by which to multiply the probabilty of secondary
            infection if `case` is asymptomatic COVID positive.
            period (int): Duration of

To do this we need some more parameters, which we also load from `config.py`. The user can, of course, specify this themselves if they would like.

In [13]:
contacts_config = config.get_contacts_config("oxteam")

In [14]:
contacts_config.keys()

dict_keys(['home_sar', 'work_sar', 'other_sar', 'period', 'asymp_factor'])

We now do the same as we did with when simulating a primary case.

In [15]:
social_contacts = simulate_contacts(primary_case, **contacts_config)
print_doc(social_contacts)

Contacts(n_daily, home, work, other)


### Deeper: Contacts attributes

_todo BE: explain contacts attributes_

All of the information about the primary case's infection and how they infect their social contacts (under no government intervention) is now contained in `primary_case` and `social_contacts`.

... Now we run a simulation, some explanation of how that works

## TTI Strategies

## Running a Simulation